In [30]:
import pandas as pd
from IPython.core.display import clear_output
from requests import get
from os.path import isfile

START_ID = 1
END_ID = 5000
CHUNKSIZE = 10

path = 'data/scraped_collegedata.csv'
if isfile(path):
    START_ID = pd.read_csv(path)['page_id'].max() + 1

df = pd.DataFrame()
for page_id in range(START_ID, END_ID + 1):
    scraped_school = {}
    for i in range(0,len(BASE_URLS)):
        url = BASE_URLS[i] + str(page_id)
        
        print(url)   # Print a status update.
        
        response = get(url)
        page = BeautifulSoup(response.text, "lxml")
        if page.h1:
            scraped_school['name'] = page.h1.string
            scraped_school.update(scrape(page, page_labels[i]))
            series = pd.Series(scraped_school, name=page_id)
            df = df.append(series)
        
        clear_output(wait = True)   # Clear the status update.
    
    if (page_id - START_ID) % CHUNKSIZE == 0:
        if not isfile(path):
            df.to_csv(path, mode='w', header=True, index_label='page_id')
        else: # else it exists so append without writing the header
            df.to_csv(path, mode='a', header=False)
        
        # Clear the dataframe.
        df = pd.DataFrame()

http://www.collegedata.com/cs/data/college/college_pg06_tmpl.jhtml?schoolId=5000


In [177]:
import numpy as np
import pandas as pd
from os.path import isfile
from requests import get
from requests_futures.sessions import FuturesSession
from IPython.core.display import clear_output
from bs4 import BeautifulSoup

# Scraper start and stop information.
BASE_URL_1 = "http://www.collegedata.com/cs/data/college/college_pg0"
BASE_URL_2 = "_tmpl.jhtml?schoolId="
SCHOOL_ID_START = 1
SCHOOL_ID_END = 5000
BATCH_SIZE = 10
EMPTY_TITLE = "Retrieve a Saved Search"
RAW_CSV_PATH = "data/scraped_collegedata.csv"
PAGE_IDS = np.arange(1, 7)


def scrape_page(soup):
    
    scraped_page = {}
    scraped_page['Name'] = soup.h1.string
    
    th_tags = soup.find(id='tabcontwrap').find_all('th')
    for th_tag in th_tags:
        if th_tag.parent:
            if th_tag.parent.td:
                key = " ".join(th_tag.stripped_strings)
                while key in scraped_page.keys():
                    key += '*'
                value = " ".join(th_tag.parent.td.stripped_strings)
                scraped_page[key] = value
                
    return scraped_page


def get_batch_reqs(batch_school_ids):
    batch_urls = {school_id: \
                  {page_id: BASE_URL_1 + str(page_id) \
                   + BASE_URL_2 + str(school_id) \
                    for page_id in PAGE_IDS} \
                     for school_id in batch_school_ids}
    
    session = FuturesSession()
    
    batch_reqs = {school_id: \
                  {page_id: session.get(batch_urls[school_id][page_id]) \
                   for page_id in PAGE_IDS} \
                    for school_id in batch_school_ids}
    
    return batch_reqs

    
    
def check_scrape_errors(soup):
    if not soup.h1:
        return True
    elif soup.h1.string == EMPTY_TITLE:
        return True
    else:
        return False




def scrape_batch(batch_start_school_id):
    batch_school_ids = np.arange(batch_start_school_id, 
                                 batch_start_school_id + BATCH_SIZE)
    batch_reqs = get_batch_reqs(batch_school_ids)
    
    scraped_batch = {}
    for school_id in batch_school_ids:
        page_reqs = batch_reqs[school_id]
        
        scraped_school = {}
        scraped_school['School Id'] = school_id
        
        for page_id, page_req in page_reqs.items():
            
            # Print a status message.
            print("Scraping page {} of school {}".format(page_id, school_id))
            
            result = page_req.result()
            
            if check_req_errors(result):
                break
                
            soup = BeautifulSoup(result.text, "lxml")
            
            if check_scrape_errors(soup):
                break
            
            scraped_page = scrape_page(soup)
            
            scraped_school.update(scraped_page)
            
            # Clear the status update.
            clear_output(wait = True)
        
        scraped_batch[school_id] = scraped_school
    
    return scraped_batch



# If already partially scraped, start where previoulsy left off.
if isfile(path):
    SCHOOL_ID_START = pd.read_csv(path)['School Id'].max() + 1

# Create numpy arrays of all requested page and school ids.
school_ids = np.arange(SCHOOL_ID_START, SCHOOL_ID_END + 1)

# Scrape the range of school ids in batches.
for batch_start_id in range(SCHOOL_ID_START, SCHOOL_ID_END + 1, BATCH_SIZE):
    
    # Scrape the current batch and convert the results to a pandas dataframe.
    scraped_batch = scrape_batch(batch_start_id)
    
    batch_df = pd.DataFrame.from_dict(scraped_batch, orient='index')
    
    # Append the dataframe to .csv:
    if not isfile(RAW_CSV_PATH):
        # If .csv file doesn't already exist, create it and include headers...
        batch_df.to_csv(RAW_CSV_PATH, mode='w', header=True, 
                        index_label='School Id')
    else:
        # ...or else just append the scraped data to the .csv without headers.
        batch_df.to_csv(RAW_CSV_PATH, mode='a', header=False)

Scraping page 1 of school 16


KeyboardInterrupt: 

In [260]:
from requests import get
URL = "https://www.collegedata.com/cs/data/college/college_pg02_tmpl.jhtml?schoolId=59"
response = get(URL)
soup = BeautifulSoup(response.text, "lxml")
soup.h1.string

'Penn State University Park'

In [505]:
import numpy as np
import pandas as pd
from os.path import isfile
from requests import get
from requests_futures.sessions import FuturesSession
from IPython.core.display import clear_output
from bs4 import BeautifulSoup
from bs4 import Comment

def get_clean_string(tag):
    child_tags = tag.find_all(True)
    if child_tags:
        for child_tag in tag.find_all(True):
            if child_tag.name == 'div':
                child_tag.unwrap()
            else:
                child_tag.decompose()     
    value = " ".join(tag.stripped_strings)
    
    return value


def scrape_row(tr_tag):
    scraped_row = {}
    
    th_tags = tr_tag.find_all('th')
    if not th_tags:
        return None
    
    th_tag = th_tags[0]
    label = get_clean_string(th_tag)
    if not label:
        return None
    
    values = []
    parent = th_tag.parent
    if not parent:
        return None
    
    td_tags = parent.find_all('td')
    if not td_tags:
        return None
    
    for td_tag in td_tags:
        value = get_clean_string(td_tag)
        values.append(value)
    
    scraped_row[label] = values
    return scraped_row

    
def scrape_table(thead_tag):
    # Get the column labels from <thead>, if they exist.
    # If they do not, use a number label placeholder instead.
    column_labels = []
    thead_td_tags = thead_tag.find_all('td')
    if not thead_td_tags:
        return None
    for thead_td_tag in thead_td_tags:
        i = 0
        if thead_td_tag.string:
            column_label = thead_td_tag.string
        else:
            column_label = None
        while column_label in column_labels:
            if column_label:
                column_label += '*'
            else:
                column_label = '*'
        column_labels.append(column_label)

    # Get the <th> strings in the <tbody>.
    if thead_tag.parent:
        if not thead_tag.parent.tbody:
            return None
        tbody_tr_tags = thead_tag.parent.tbody.find_all('tr')
        if not tbody_tr_tags:
            return None
    
    scraped_table = {}
    # Scrape the rows of each table.
    for tr_tag in tbody_tr_tags:
        scraped_row = {}
        raw_scraped_row = scrape_row(tr_tag)
        if raw_scraped_row:
            all_raw_values = []
            for raw_value in raw_scraped_row.values():
                all_raw_values += raw_value
            unique_vals = set(all_raw_values)
                
            for raw_label, raw_values in raw_scraped_row.items():
                for i, raw_value in enumerate(raw_values):
                    if (len(unique_vals) == 2) and ('X' in unique_vals):
                        if raw_value == 'X':
                            scraped_row[raw_label] = column_labels[i]
                    else:
                        column_label = column_labels[i]
                        label = raw_label
                        if column_label:
                            label = raw_label + " - " + column_label
                        scraped_row[label] = raw_value
                        
        scraped_table.update(scraped_row)
        
    return scraped_table


def remove_sports_offered_table(soup):
    comment_tag = soup.find(string=" Sports start ")
    if not comment_tag:
        return None
    table_tag = comment_tag.find_next_sibling('table')
    if not table_tag:
        return None
    table_tag.decompose()
    return None


def fix_population_label(soup):
    regexp = re.compile('Population')
    th_tags = soup.find_all(string=regexp)
    if not th_tags:
        return None
    for th_tag in th_tags:
        th_tag.string.replace_with('City Population')
    return None


def remove_section(soup, section_id):
    tag = soup.find(id = section_id)
    if not tag:
        return None
    tag.decompose()
    return None


def preprocess_soup(soup):
    remove_sports_offered_table(soup)
    fix_population_label(soup)
    remove_section(soup, "section10")
    return None


def add_dict(dict_1, dict_2):
    if dict_2:
        temp_dict = {}
        for key, value in dict_2.items():
            if key not in dict_1.keys():
                temp_dict[key] = value
            elif value != dict_1[key]:
                while key in dict_1.keys():
                    key += '*'
                temp_dict[key] = value
        dict_1.update(temp_dict)
    return None
        

def scrape_page(soup):    
    
    scraped_page = {}

    # Scrape data from tables with <thead> and <tbody> tags, if any.
    thead_tags = soup.find(id='tabcontwrap').find_all('thead')
    if thead_tags:
        for thead_tag in thead_tags:
            scraped_table = scrape_table(thead_tag)
            add_dict(scraped_page, scraped_table)
        
            # Delete entire table from soup.
            thead_tag.parent.decompose()
        

    # Scrape remaining table rows.
    tr_tags = soup.find(id='tabcontwrap').find_all('tr')
    if tr_tags:
        for tr_tag in tr_tags:
            scraped_row = scrape_row(tr_tag)
            if scraped_row:
                for label, values in scraped_row.items():
                    scraped_row[label] = values[0]
            
                add_dict(scraped_page, scraped_row)
    
    return scraped_page


In [540]:
BASE_URL_1 = "https://www.collegedata.com/cs/data/college/college_pg0"
BASE_URL_2 = "_tmpl.jhtml?schoolId="
PAGE_IDS = np.arange(1, 7)
SCHOOL_IDS = np.arange(1, 5000)
BATCH_SIZE = 1

weird_soups = []
errors = []
def scrape_batch(batch_start):
    batch = SCHOOL_IDS[batch_start : batch_start + BATCH_SIZE]
    
    urls = {school_id: \
            {page_id:BASE_URL_1 + str(page_id) + BASE_URL_2 + str(school_id) \
              for page_id in PAGE_IDS} \
               for school_id in batch}

    session = FuturesSession()
    reqs = {school_id: [session.get(url) for url in urls[school_id].values()] \
             for school_id in batch}

    
    scraped_batch = {}
    for school_id in batch:
        scraped_school = {}
        name = None
        for req in reqs[school_id]:
            clear_output(wait = True)
            result = req.result()
            print("Scraping {}".format(result.url))
            print("Status Code: {}".format(result.status_code))
            
            if result.status_code != 200:
                error = "Status Code {}".format(result.status_code)
                print("ERROR {} with {}".format(error, result.url))
                errors.append((result.url, result.status_code))
                break
            
            soup = BeautifulSoup(result.text, "lxml")
            
            if not soup.h1:
                error = "No <h1> tag!"
                print("ERROR {} with {}".format(error, result.url))
                errors.append((result.url, error))
                break
            
            if soup.h1.string == EMPTY_TITLE:
                error = "Empty page!"
                print("ERROR {} with {}".format(error, result.url))
                errors.append((result.url, error))
                break
                
            preprocess_soup(soup)
            scraped_page = scrape_page(soup)
            
            if len(scraped_page) == 1:
                weird_soups.append(soup)
            
            
            name = soup.h1.string
            
            add_dict(scraped_school, scraped_page)
            
        scraped_school['Name'] = name
        scraped_batch[school_id] = scraped_school
    
    return scraped_batch



SCHOOL_ID_END = 20
scraped = {}
for batch_start in range(0, SCHOOL_ID_END + 1, BATCH_SIZE):
    scraped_batch = scrape_batch(batch_start)
    scraped.update(scraped_batch)

df = pd.DataFrame.from_dict(scraped, orient='index')

df

Scraping https://www.collegedata.com/cs/data/college/college_pg06_tmpl.jhtml?schoolId=21
Status Code: 200


,Name,Academic GPA,Standardized Tests,Essay,Extracurricular Activities,Web Site,Institution Type,Coeducational,Undergraduate Students,Women,...,Top Areas (By Money Awarded),Creative Arts/Performance Award Areas,Special Achievements/Activities Award Areas,Special Characteristics Award Areas,Work-Study Programs,Average Earnings from On-Campus Employment,Number of Awards*,Number of Awards**,Number of Awards***,Coeducational*
1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bryn Athyn College,Very Important,Very Important,Very Important,Considered,,Private,Yes,326,165 (50.6%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Adelphi University,Important,Important,Important,Important,,Private,Yes,"5,266","3,603 (68.4%)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Albany College of Pharmacy and Health Sciences,Very Important,Very Important,Important,Important,,Private,Yes,"1,078",646 (59.9%),...,,Not reported,Not reported,"Children and Siblings of Alumni, Children of F...","Federal work study available, other work study...",Not reported,NaN,NaN,NaN,NaN
9,Albertus Magnus College,Very Important,Considered,Important,Considered,,Private,Yes,"1,220",814 (66.7%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Albright College,Very Important,Considered,Considered,Considered,,Private,Yes,"2,015","1,195 (59.3%)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [541]:
len(weird_soups)

8

In [543]:
soup = weird_soups[0]

#soup('h1')[0].parent.parent.next_sibling.next_sibling.next_sibling.next_sibling

soup.h1.string

'Bryn Athyn College'

In [513]:
df['Institution Type']

2         NaN
3         NaN
4         NaN
5         NaN
6     Private
7     Private
8     Private
9     Private
10    Private
11    Private
12    Private
13    Private
14    Private
15    Private
16        NaN
17    Private
18    Private
19    Private
20    Private
21    Private
Name: Institution Type, dtype: object

In [488]:
result = get('http://www.vertuli.com')

In [492]:
result.status_code

200